# TASK 2.1 

In [3]:
pip install dvc

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install mlflow


     ---------------------------------------- 19.1/19.1 MB 6.8 MB/s eta 0:00:00
     -------------------------------------- 413.4/413.4 kB 8.6 MB/s eta 0:00:00
     ---------------------------------------- 24.6/24.6 MB 6.7 MB/s eta 0:00:00
     ---------------------------------------- 41.2/41.2 kB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 57.7/57.7 kB 3.0 MB/s eta 0:00:00
     -------------------------------------- 150.3/150.3 kB 4.5 MB/s eta 0:00:00
     -------------------------------------- 148.1/148.1 kB 4.3 MB/s eta 0:00:00
     -------------------------------------- 233.4/233.4 kB 7.2 MB/s eta 0:00:00
     ---------------------------------------- 78.6/78.6 kB ? eta 0:00:00
     -------------------------------------- 151.7/151.7 kB 2.3 MB/s eta 0:00:00


In [100]:
store_df = pd.read_csv("store.csv")

train_df = pd.read_csv("train.csv")

test_df = pd.read_csv("test.csv")

In [102]:
train_store_merged=pd.merge(train_df,store_df)
train_store_merged.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [103]:
import os
os.environ["NUMEXPR_MAX_THREADS"] = "4"  # Set the number of threads you prefer

import warnings

warnings.filterwarnings("ignore")

import os
import logging
logging.basicConfig(level=os.environ.get("LOGLEVEL", "INFO"))

import pandas as pd
import dvc.api
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Display max column
pd.set_option('display.max_columns', None)
## Display max rows
# pd.set_option('display.max_rows', None)

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor  

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import mlflow
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error

import pickle # Read and Save model - Serialization
from datetime import datetime

In [104]:
#Logger
log = logging.getLogger()
log.info('check')

INFO:root:check


## Preprocessing

In [105]:
## loading merged train,store data using pandas library
train_store_df = pd.read_csv("train_store_merged.csv",low_memory = False)
train_store_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,WeekOfYear,WeekofMonth,SalePerCustomer,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,2015,7,31,31,5,9.482883,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
1,1,4,2015-07-30,5020,546,1,1,0,1,2015,7,30,31,5,9.194139,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
2,1,3,2015-07-29,4782,523,1,1,0,1,2015,7,29,31,5,9.143403,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
3,1,2,2015-07-28,5011,560,1,1,0,1,2015,7,28,31,5,8.948214,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
4,1,1,2015-07-27,6102,612,1,1,0,1,2015,7,27,31,4,9.970588,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0


In [106]:
## getting number of data points in the data
print(f"There are {train_store_df.shape[0]} rows and {train_store_df.shape[1]} columns in the train_store dataset")

There are 1017209 rows and 24 columns in the train_store dataset


In [108]:
train_store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 24 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  int64  
 1   DayOfWeek                  1017209 non-null  int64  
 2   Date                       1017209 non-null  object 
 3   Sales                      1017209 non-null  int64  
 4   Customers                  1017209 non-null  int64  
 5   Open                       1017209 non-null  int64  
 6   Promo                      1017209 non-null  int64  
 7   StateHoliday               1017209 non-null  object 
 8   SchoolHoliday              1017209 non-null  int64  
 9   Year                       1017209 non-null  int64  
 10  Month                      1017209 non-null  int64  
 11  Day                        1017209 non-null  int64  
 12  WeekOfYear                 1017209 non-null  int64  
 13  WeekofMonth 

In [109]:
## Convert the 'Date' column to datetime format
train_store_df['Date'] = pd.to_datetime(train_store_df['Date'])

In [110]:
train_store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 24 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   Store                      1017209 non-null  int64         
 1   DayOfWeek                  1017209 non-null  int64         
 2   Date                       1017209 non-null  datetime64[ns]
 3   Sales                      1017209 non-null  int64         
 4   Customers                  1017209 non-null  int64         
 5   Open                       1017209 non-null  int64         
 6   Promo                      1017209 non-null  int64         
 7   StateHoliday               1017209 non-null  object        
 8   SchoolHoliday              1017209 non-null  int64         
 9   Year                       1017209 non-null  int64         
 10  Month                      1017209 non-null  int64         
 11  Day                        1017209 no

In [121]:
## columns to use
columns = ['DayOfWeek', 'Date', 'Open', 'Promo', 'StateHoliday',
       'SchoolHoliday', 'Year', 'Month', 'Day', 'WeekOfYear']

# Label Date and StateHoliday Train
columns = ['Sales', 'Store', 'DayOfWeek', 'Open', 'Promo', 'SchoolHoliday', 'StateHoliday', 'Day', 'WeekOfYear', 
           'WeekofMonth', 'Month', 'Year', 'StoreType', 'Assortment', 'CompetitionDistance', 'Promo2']

feature_columns = ['Store', 'DayOfWeek', 'Open', 'Promo', 'SchoolHoliday', 'StateHoliday', 'Day', 'WeekOfYear', 
                   'WeekofMonth', 'Month', 'Year', 'StoreType', 'Assortment', 'CompetitionDistance', 'Promo2']

sample_size = 1000
sampled_df = train_store_df[columns].sample(sample_size)

# Separate Test and Train files
train_x = sampled_df[feature_columns]
train_y = sampled_df[['Sales']]
train_x.head()

,Store,DayOfWeek,Open,Promo,SchoolHoliday,StateHoliday,Day,WeekOfYear,WeekofMonth,Month,Year,StoreType,Assortment,CompetitionDistance,Promo2
204261,226,6,1,0,0,0,20,51,3,12,2014,a,a,450.0,0
240057,265,2,1,1,0,0,14,29,3,7,2015,a,a,4580.0,1
28800,32,2,1,1,0,0,3,10,1,3,2015,a,a,2910.0,1
694598,763,2,1,1,1,0,19,34,3,8,2014,d,c,32240.0,0
849259,933,7,0,0,0,0,12,28,2,7,2015,a,c,6270.0,0


In [122]:
### Label encoding
label_encoder = LabelEncoder()
train_x['StoreType'] = label_encoder.fit_transform(train_x['StoreType']) ## Encoding the 'StoreType' column using label encoding
train_x['Assortment'] = label_encoder.fit_transform(train_x['Assortment']) ## Encoding the 'Assortment' column using label encoding
train_x['StateHoliday'] = label_encoder.fit_transform(train_x['StateHoliday']) ## Encoding the 'StateHoliday' column using label encoding

train_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 204261 to 451923
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Store                1000 non-null   int64  
 1   DayOfWeek            1000 non-null   int64  
 2   Open                 1000 non-null   int64  
 3   Promo                1000 non-null   int64  
 4   SchoolHoliday        1000 non-null   int64  
 5   StateHoliday         1000 non-null   int32  
 6   Day                  1000 non-null   int64  
 7   WeekOfYear           1000 non-null   int64  
 8   WeekofMonth          1000 non-null   int64  
 9   Month                1000 non-null   int64  
 10  Year                 1000 non-null   int64  
 11  StoreType            1000 non-null   int32  
 12  Assortment           1000 non-null   int32  
 13  CompetitionDistance  1000 non-null   float64
 14  Promo2               1000 non-null   int64  
dtypes: float64(1), int32(3), int64(

In [123]:
# Initialize the scalers
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

# Scale the features
X = x_scaler.fit_transform(train_x)
X = train_x
# Scale the target variable
Y = y_scaler.fit_transform(train_y)

In [124]:
X.shape, Y.shape

((1000, 15), (1000, 1))

In [125]:
split_size = int(0.8 * len(X))  ## Splitting the dataset into 80% for training and 20% for testing

X_train, y_train = X[:split_size], Y[:split_size]  ## Assigning the first 80% of the data to the training set

X_test, y_test = X[split_size:], Y[split_size:]  ## Assigning the remaining 20% of the data to the testing set

len(X_train), len(X_test), len(y_train), len(y_test)  ## Printing the lengths of the training and testing sets

(800, 200, 800, 200)

In [126]:
## Create a random forest regressor object 
rf_reg = RandomForestRegressor(n_estimators = 100, max_depth=25, random_state=42)
rf_reg.fit(X_train, y_train)

RandomForestRegressor(max_depth=25, random_state=42)

In [131]:
## Calculate the prediction score of the model
score = rf_reg.score(X_test, y_test)
print(f"Prediction Score of the Model is {round(score * 100, 2)}%")

Prediction Score of the Model is 56.08%


# CONCLUSION

# Prediction Score of the Model is 56.08%